In [1]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
import pickle
import string

In [11]:
fund_name = 't_rowe_price'


CIK_LIST = ['313212', '852254', '902259', '1177017', '1034386', '918294', '920467', '754915', '1120925', '1533490', '923084', '202927', '853437', '927845', '871839', '931151', '949820', '894024', '1019286', '1002624', '858581', '1181628', '1169187', '918292', '887147', '80248', '80249', '731890', '75170', '834798','1707770','1681576']



In [12]:
cur_wd = os.getcwd()
if 'trunk' in cur_wd:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output')
else:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output','working')
    
cached_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'cached_data')
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print('data directory')
print(cur_wd)
print('output directory')
print(output_directory)

print('cached data')
print(cached_wd)


data directory
C:\Users\jjanko\Desktop\wei_mutual_fund_project\data
output directory
C:\Users\jjanko\Desktop\wei_mutual_fund_project\output\working
cached data
C:\Users\jjanko\Desktop\wei_mutual_fund_project\cached_data


# Helper Functions for Analysis

In [13]:
def get_cleaned(lines):
    cleaned = []
    for i in lines:
        #gets rid of tags
        cleaned_i = re.sub('<[^>]*>', ' ', i)
        #gets rid of &nbsp; and replaces with space
        cleaned_i = re.sub('&nbsp;', ' ', cleaned_i)
        #re.sub("<.*?>","",st)
        if not re.match(r'^\s*$', cleaned_i):
            cleaned.append(cleaned_i)
    return cleaned

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))


def remove_junk(addline):
    addline = re.sub('<[^>]*>', ' ', addline)
    addline = re.sub('&nbsp;', ' ', addline)
    addine = re.sub('&#174;', ' ', addline)
    addline = re.sub('&#8480;', ' ', addline)
    addline = re.sub(r'&reg;',' ',addline)
    addline = re.sub(r'\n', ' ', addline)
    addline = re.sub(r'\t', ' ', addline)
    return addline

def getDuplicateColumns(df): 
  
    # Create an empty set 
    duplicateColumnNames = set() 
      
    # Iterate through all the columns  
    # of dataframe 
    for x in range(df.shape[1]): 
          
        # Take column at xth index. 
        col = df.iloc[:, x] 
          
        # Iterate through all the columns in 
        # DataFrame from (x + 1)th index to 
        # last index 
        for y in range(x + 1, df.shape[1]): 
              
            # Take column at yth index. 
            otherCol = df.iloc[:, y] 
              
            # Check if two columns at x & y 
            # index are equal or not, 
            # if equal then adding  
            # to the set 
            if col.equals(otherCol): 
                duplicateColumnNames.add(df.columns.values[y]) 
                  
    # Return list of unique column names  
    # whose contents are duplicates. 
    return list(duplicateColumnNames) 

In [14]:
with open(os.path.join(cached_wd, 'errors_dict.pickle'), 'rb') as handle:
    errors = pickle.load(handle)
    
print(errors)

        

{'867297': ['0001193125-04-202597.txt', '0001193125-05-115532.txt', '0001193125-05-232633.txt'], '1091439': ['0000950152-04-008622.txt', '0000950152-05-004867.txt', '0000950152-05-009599.txt'], '1301708': ['0000950152-05-009600.txt'], '1423227': [], '1098605': ['0000950136-05-006141.txt', '0000950136-04-003174.txt', '0000950136-05-001871.txt'], '1378240': ['0001104659-07-029038.txt', '0001104659-07-077264.txt', '0001104659-08-027201.txt', '0001104659-08-064591.txt'], '726735': ['0000950129-04-009378.txt', '0000950129-05-005810.txt', '0000950129-05-011455.txt'], '844779': ['0001193125-05-039836.txt', '0001193125-05-175146.txt', '0001193125-05-175147.txt', '0001193125-05-175620.txt'], '1398078': [], '774013': ['0001193805-05-000319.txt', '0001193805-05-001781.txt', '0001193805-05-002628.txt', '0001193805-05-002636.txt', '0001193805-05-002638.txt'], '1062806': ['0000900092-04-000310.txt', '0000900092-05-000103.txt', '0000900092-05-000135.txt', '0000900092-05-000453.txt', '0001193805-04-00

In [15]:
weblink = {}
txt = []

for CIK in CIK_LIST:
    weblink[CIK]  = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        txt.append(i)
        weblink[CIK].append(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK)+r"/"  + i)

In [16]:
with open(os.path.join(cached_wd, 'panel2.pickle'), 'rb') as handle:
    panel2 = pickle.load(handle)
    


In [17]:
panel2 = panel2.loc[panel2['file_read'].isin(txt)]

In [18]:
print(panel2)
panel2.to_csv(os.path.join(output_directory, fund_name + '_panel2.csv'), sep = ',')

   CLASS-CONTRACT-ID                                CLASS-CONTRACT-NAME  \
0         C000004003             T. Rowe Price International Stock Fund   
1         C000004004  T. Rowe Price International Stock Fund-Advisor...   
2         C000004005     T. Rowe Price International Stock Fund-R Class   
3         C000004006  T. Rowe Price Emerging Europe & Mediterranean ...   
4         C000004010          T.Rowe Price International Discovery Fund   
5         C000004011                  T. Rowe Price European Stock Fund   
6         C000004012                        T. Rowe Price New Asia Fund   
7         C000004013                           T. Rowe Price Japan Fund   
8         C000004014                   T. Rowe Price Latin America Fund   
9         C000004015          T. Rowe Price Emerging Markets Stock Fund   
10        C000004016                    T. Rowe Price Global Stock Fund   
11        C000004017   T. Rowe Price International Growth & Income Fund   
12        C000004018  T. 

## Test on One FILE

In [19]:

CIK  ='1681576'
text =  '0001206774-19-001529.txt'


In [20]:
panel2.loc[panel2['file_read'] == text]

,CLASS-CONTRACT-ID,CLASS-CONTRACT-NAME,CLASS-CONTRACT-TICKER-SYMBOL,SERIES-ID,OWNER-CIK,SERIES-NAME,file_read,date_filed,company conformed name,reporting_date
0,C000174964,T. Rowe Price Total Return Fund,PTTFX,S000055582,0001681576,T. Rowe Price Total Return Fund,0001206774-19-001529.txt,20190429,"T. Rowe Price Total Return Fund, Inc.",20190228
1,C000174965,T. Rowe Price Total Return Fund--Advisor Class,PTATX,S000055582,0001681576,T. Rowe Price Total Return Fund,0001206774-19-001529.txt,20190429,"T. Rowe Price Total Return Fund, Inc.",20190228
2,C000174966,T. Rowe Price Total Return Fund--I Class,PTKIX,S000055582,0001681576,T. Rowe Price Total Return Fund,0001206774-19-001529.txt,20190429,"T. Rowe Price Total Return Fund, Inc.",20190228


In [21]:
names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
names = names.unique()
print(names)

['T. Rowe Price Total Return Fund']


In [22]:
url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
html_table = urlopen(url).read()
soup = BeautifulSoup(html_table, "html.parser")
tags = soup.find_all(['table','p'])

In [23]:
print(url)

https://www.sec.gov/Archives/edgar/data/1681576/000120677419001529/0001206774-19-001529.txt


In [24]:
tables = []
iterate = 0
for i in tags:
    for n in names:
        if n.lower() in str(i).lower().replace('\n',' '):
            fundname = n
            print(fundname)
            
    try:
        df = pd.read_html(str(i), flavor="bs4")
        for j in df:
            if len(j.index) > 0 :
                tables.append(j)
                iterate = iterate + 1
    except:
        pass

T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund


In [25]:
url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
html_table = urlopen(url).read()
soup = BeautifulSoup(html_table, "html.parser")
tags = soup.find_all(['table'])

In [31]:
print(url)

https://www.sec.gov/Archives/edgar/data/1681576/000120677419001529/0001206774-19-001529.txt


In [26]:
tables = []

for i in tags:
    for n in names:
        if n.lower() in str(i).lower().replace('\n',' '):
            fundname = n
            print(fundname)
            
    try:
        df = pd.read_html(str(i), flavor="bs4")
        for j in df:
            if len(j.index) > 0 :
                tables.append(j)
    except:
        pass

T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund
T. Rowe Price Total Return Fund


In [30]:
for i in tables:
    print(i)

                                                   0
0              T. Rowe Price Total Return Fund, Inc.
1  (Exact name of registrant as specified in char...
2         100 East Pratt Street, Baltimore, MD 21202
3           (Address of principal executive offices)
4                                  David Oestreicher
5         100 East Pratt Street, Baltimore, MD 21202
6            (Name and address of agent for service)
                   0                  1
0  Total Return Fund  February 28, 2019
                                 0
0  T. ROWE PRICE TOTAL RETURN FUND
                                   0
0  NOTES TO PORTFOLIO OF INVESTMENTS
                     0    1                            2
0                  NaN   By        /s/ David Oestreicher
1                  NaN  NaN            David Oestreicher
2                  NaN  NaN  Principal Executive Officer
3                  NaN  NaN                          NaN
4  Date April 24, 2019  NaN                          NaN
          